In [2]:
from alpaca.data.live import CryptoDataStream, StockDataStream
import Authorization
import asyncio

In [3]:
# Asign Key and secret value
API_KEY = Authorization.ALPACA_API_KEY
SECRET_KEY = Authorization.ALPACA_SECRET_KEY
# Initiate class
crypto_stream = CryptoDataStream(API_KEY, SECRET_KEY)
Stock_stream = StockDataStream(API_KEY, SECRET_KEY)

In [7]:
# define callback function to print the bar upon reciving.
async def bar_callback(bar):
    for property_name, value in bar:
        print(f"\"{property_name}\": {value}")
        
# Subscribing to bar event 
symbol = "BTC/USD"
crypto_stream.subscribe_bars(bar_callback, symbol)
crypto_stream.run()

RuntimeError: asyncio.run() cannot be called from a running event loop

In [4]:
from alpaca.data.live import StockDataStream

In [5]:
stock_stream = StockDataStream(API_KEY, SECRET_KEY)

In [6]:
# async handler
async def quote_data_handler(data: any):
    # quote data will arrive here
    print(data)
    
symbols = 'TSLA'
stock_stream.subscribe_quotes(quote_data_handler,symbols)

stock_stream.run()

RuntimeError: asyncio.run() cannot be called from a running event loop

In [4]:
from websocket import create_connection
import simplejson as json
import pprint

url = 'wss://stream.data.alpaca.markets/v2/iex'
ws = create_connection(url)
print(json.loads(ws.recv())) #print the connection message

auth_message = {"action": "auth", "key": API_KEY, "secret": SECRET_KEY}
ws.send(json.dumps(auth_message))
print(json.loads(ws.recv())) # print the authenticated message

subscription = {"action":"subscribe","trades":["AAPL"],"quotes":["AMD","CLDR"],"bars":["AAPL","VOO"]}
ws.send(json.dumps(subscription))
print(json.loads(ws.recv())) # print the subscribed message

while True:
    data = json.loads(ws.recv())
    pprint.pprint(data[0])
    print('*********************')

[{'T': 'success', 'msg': 'connected'}]
[{'T': 'success', 'msg': 'authenticated'}]
[{'T': 'subscription', 'trades': ['AAPL'], 'quotes': ['AMD', 'CLDR'], 'bars': ['AAPL', 'VOO'], 'updatedBars': [], 'dailyBars': [], 'statuses': [], 'lulds': [], 'corrections': ['AAPL'], 'cancelErrors': ['AAPL']}]


KeyboardInterrupt: 